In [1]:
import torch
import warnings

from tqdm import tqdm

from heart_desease_dataset import HeartDeseaseDataset

from mp_model import MPModel

warnings.filterwarnings("ignore")

In [2]:
dataset_train = HeartDeseaseDataset('dataset', train=True)
train_data_loader = torch.utils.data.DataLoader(dataset_train, batch_size=8, shuffle=True, num_workers=4)

device = 'mps'

model = MPModel(in_size=30, n_classes=2)
model.to(device)
model.train()


criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) 

total_epochs = 100

for epoch in range(total_epochs):
    # it_pbar = tqdm(enumerate(train_data_loader), desc='')
    for idx, batch in enumerate(train_data_loader):
        
        optimizer.zero_grad()  
        
        x, y = batch

        x = x.to(device)
        y = y.to(device)

        y_pred = model(x)

        loss = criterion(y_pred, y)

        loss.backward()

        optimizer.step()

    print(f'Epoch: {epoch}, it {idx}, loss: {loss.item()}')

Epoch: 0, it 56, loss: 0.26199495792388916
Epoch: 1, it 56, loss: 0.4517425000667572
Epoch: 2, it 56, loss: 0.4124743938446045
Epoch: 3, it 56, loss: 0.3021504580974579
Epoch: 4, it 56, loss: 0.2033325433731079
Epoch: 5, it 56, loss: 0.22402368485927582
Epoch: 6, it 56, loss: 0.2531684637069702
Epoch: 7, it 56, loss: 0.05162167549133301
Epoch: 8, it 56, loss: 0.06957464665174484
Epoch: 9, it 56, loss: 0.3176189959049225
Epoch: 10, it 56, loss: 0.23719382286071777
Epoch: 11, it 56, loss: 0.0141450809314847
Epoch: 12, it 56, loss: 0.04735295847058296
Epoch: 13, it 56, loss: 0.07966816425323486
Epoch: 14, it 56, loss: 0.04112572968006134
Epoch: 15, it 56, loss: 0.006863874848932028
Epoch: 16, it 56, loss: 0.008787834085524082
Epoch: 17, it 56, loss: 0.15911220014095306
Epoch: 18, it 56, loss: 0.06267353147268295
Epoch: 19, it 56, loss: 8.841188900987618e-06
Epoch: 20, it 56, loss: 0.14271168410778046
Epoch: 21, it 56, loss: 0.025151953101158142
Epoch: 22, it 56, loss: 0.04091561958193779


In [3]:
test_dataset    = HeartDeseaseDataset('dataset', train=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=True, num_workers=2)

model.eval()

corret_samples = 0
for x_batch, y_batch in tqdm(test_dataloader):
    x = x.to(device)
    y = y.to(device)
    x, y = batch
    with torch.no_grad():
        y_hat = model(x)

    corret_samples += y_hat.argmax(dim=1).eq(y).sum()

print(f'Final Acurancy on test set is: {corret_samples/len(test_dataset)*100}%' )


  0%|          | 0/19 [00:06<?, ?it/s]


RuntimeError: Placeholder storage has not been allocated on MPS device!